#  About Clubhouse

**Clubhouse** is an invitation-only social media app for iOS that facilitates auditory communication through rooms that can accommodate groups of up to 5,000 people.The audio-only app hosts virtual rooms for live discussions, with opportunities for individuals to participate through speaking and listening.

# In this notebook

There are 1300515 rows in clubhouse dataset table. Columns in clubhouse dataset:
1. **user_id**
2. **name**
3. **photo_url**
4. **username**
5. **twitter**
6. **instagram**
7. **num_followers**
8. **num_following**
9. **time_created**
10. **invited_by_user_profile**

With seaborn and plotly library, I will visualize from the dataset to answer these questions:
1. From column **time_created**, in what months are the new accounts created the most?
2. From column **num_followers**, how is the distribution of the users' followers?
3. From column **num_followers**, which users (Top 20) have the most followers? 
4. From column **num_following**, which users (Top 10) have the largest number of following? 
5. From column **invited_by_user_profile**, which users (Top 10) have invited new users the most? 
6. From column  **twitter** & **instagram**, what are the percentage of clubhouse users who have twitter or instagram account?



In [ ]:
#Import libraries
import sqlite3
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Import dataset
conn = sqlite3.connect('../input/clubhouse-dataset/Clubhouse_Dataset_v1.db')
df = pd.read_sql_query('SELECT * FROM user', conn)

#Filling missing value
df = df.fillna('')

### 1. What months are the new accounts created the most?

In [ ]:
#Convert time_created column to date format
df['time_created'] = pd.to_datetime(df['time_created']).apply(lambda x: x.date())
df['time_created']

#Creating dataframe for new accounts created
new_user_df = df.groupby('time_created', as_index=False).agg(NewUser=('time_created','count'))

#Creating line plot
plt.figure(figsize=(12,8))
sns.set_style("darkgrid")
sns.lineplot(x=new_user_df['time_created'], y=new_user_df['NewUser'])
plt.xlabel('Datetime', fontsize=14)
plt.ylabel('New User', fontsize=14)
plt.title('Lineplot of Account Creation Datetime', fontsize=16)
plt.show()

From the line plot, it is shown that between December 2020 to January 2021 were the time when new accounts were created the most.

### 2. How is the distribution of the users' followers?

In [ ]:
df['num_followers'].describe()

From the describe() function above, we know that 75% (Q3) of users have less than or equal 99 followers. From that information, I divided users' followers numbers into 3 groups for visualization so later we know their proportion:
1. Group 1: Less than 100 followers
2. Group 2: Followers between 100 and 1000
3. Group 3: More than 1000 followers 

In [ ]:
group_1 = round((df['num_followers']<100).sum()/len(df)*100)
group_2 = round(((df['num_followers']>=100)&(df['num_followers']<=1000)).sum()/len(df)*100)
group_3 = round((df['num_followers']>1000).sum()/len(df)*100)


fol_dict = {'Group':["Less than 100 followers", "Followers between 100 and 1000", 'More than 1000 followers'],
            'Group Percentage':[group_1, group_2, group_3]}
# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=fol_dict['Group'], values = fol_dict['Group Percentage'], hole=.4)])

fig.update_layout(annotations=[dict(text='Followers', x=0.5,y=0.5, font_size=20, showarrow=False)])
fig.show()

### 3. Which users (Top 20) have the most followers?

In [ ]:
largest_followers = df.nlargest(20,['num_followers'])

plt.figure(figsize=(10,8))
fol_plot = sns.barplot(x=largest_followers['num_followers'], y = largest_followers['name'], palette='mako')
plt.xlabel('Followers', fontsize=14)
plt.ylabel('User', fontsize=14)
plt.title('Top 20 Most Followed Users', fontsize=16)
plt.show()

### 4. Which users (Top 10) have the largest number of following?

In [ ]:
most_following = df.nlargest(10,['num_following'])

plt.figure(figsize=(10,7))
f_plot = sns.barplot(x= most_following['name'], y=most_following['num_following'], color='gold', edgecolor='black')
plt.xlabel('User', fontsize=16)
plt.ylabel('Following Numbers', fontsize=16)
plt.xticks(rotation=30)
plt.title('Users with Largest Number of Following', fontsize=18)
for p in f_plot.patches:
    f_plot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.show()

###  5. Which users (Top 10) have invited new users the most?

In [ ]:
#Create dataframe for top invitee
invite_df = pd.DataFrame(df.invited_by_user_profile.value_counts())
invite_df.reset_index(level=0,inplace=True)
invite_df.columns = ['user_id','invites count']
top_invitee = invite_df.iloc[:10,:]

invites_count = top_invitee['invites count']
user_list=[]
for i in top_invitee.index:
    user_name = df['name'].loc[(df['user_id']==top_invitee.loc[i,'user_id'])].values
    if top_invitee.loc[i,'user_id']!='null':
        if user_name.size!=0:
            user_list.append(user_name[0])
        else:
            user_list.append('User_id ' + str(top_invitee.loc[i,'user_id']))       
    else:
         user_list.append('No Invitee')

top_invitee['User Name'] = user_list
top_invitee.drop(['user_id','invites count'],axis=1,inplace=True)
top_invitee['invites count'] = invites_count

#Visualization
plt.figure(figsize=(10,7))
inv_plot = sns.barplot(x = top_invitee['User Name'], y=top_invitee['invites count'], palette='viridis')
plt.xlabel('User', fontsize=14)
plt.ylabel('Number of invites', fontsize=14)
plt.xticks(rotation=30)
plt.title('Top 10 Users with Most Invites', fontsize=14)
for p in inv_plot.patches:
    inv_plot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.show()

Although to join clubhouse, new user has to be invited by existing user but there are 3214 users who have joined without being invited by other users.

### 6. What are the percentage of clubhouse users who have twitter or instagram account?

In [ ]:
no_twitter_acc_perc =  (df['twitter']=='null').sum()/len(df)*100
no_insta_acc_perc = (df['instagram']=='null').sum()/len(df)*100

soc_med_dict = {'Account':["Do not have account", "Have Account"],'Twitter':[no_twitter_acc_perc, (100-no_twitter_acc_perc)],
                'Instagram':[no_insta_acc_perc, (100-no_insta_acc_perc)]}

#Visualization
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=soc_med_dict['Account'], values=soc_med_dict['Twitter'], name="Twitter"),
              1, 1)
fig.add_trace(go.Pie(labels=soc_med_dict['Account'], values=soc_med_dict['Instagram'], name="Instagram"),
              1, 2)

# Use `hole` to create a donut-like pie chart
colors = ['orangered','royalblue']
fig.update_traces(hole=.4, hoverinfo="label+percent+name",  marker=dict(colors=colors))

fig.update_layout(
    title_text="Social Media Account",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Twitter', x=0.19,y=0.5, font_size=14, showarrow=False),
                 dict(text='Instagram', x=0.81, y=0.5, font_size=14, showarrow=False)])
fig.show()

From the pie chart, we know that most of the clubhouse users (more than 50 %) don't have Twitter or Instagram account